Import relevant libraries 

In [1]:
import pandas as pd
import numpy as np
import theano
import theano.tensor as t


# Import and Split the Data

In [2]:
dataset = pd.read_csv("auto-mpg.csv", na_values="?", dtype=theano.config.floatX)
dataset.dropna(inplace=True)
# dataset = dataset.set_index(['name'])
print dataset.head()


   mpg  cylinders  displacement  horsepower  weight   acceleration  year  \
0   18          8           307         130     3504          12.0    70   
1   15          8           350         165     3693          11.5    70   
2   18          8           318         150     3436          11.0    70   
3   16          8           304         150     3433          12.0    70   
4   17          8           302         140     3449          10.5    70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  

[5 rows x 8 columns]


In [3]:
print dataset.shape
print dataset.values

(392, 8)
[[  18.     8.   307.  ...,   12.    70.     1. ]
 [  15.     8.   350.  ...,   11.5   70.     1. ]
 [  18.     8.   318.  ...,   11.    70.     1. ]
 ..., 
 [  32.     4.   135.  ...,   11.6   82.     1. ]
 [  28.     4.   120.  ...,   18.6   82.     1. ]
 [  31.     4.   119.  ...,   19.4   82.     1. ]]


In [4]:
train = dataset.values[0:250, :]
print train.shape
print train[:10, 0]
test = dataset.values[250:, :]

train_x = train[:, 1:]
train_y = train[:, 0]
test_x = test[:, 1:]
test_y = test[:, 0]

print train_y[:10]
print train_x.shape

(250, 8)
[ 18.  15.  18.  16.  17.  15.  14.  14.  14.  15.]
[ 18.  15.  18.  16.  17.  15.  14.  14.  14.  15.]
(250, 7)



# Set up the theano variables 

In [5]:
x = t.dmatrix("X")
y = t.dvector("y")

w_value = np.zeros(7, dtype=theano.config.floatX)
b_value = np.asarray(0., dtype=theano.config.floatX)


w = theano.shared(value=w_value, name="weights")
b = theano.shared(value=b_value, name="bias")
params = [w, b]

## Build our functions 

In [8]:
# Build Hypothesis
lin_hypothesis = t.dot(x, w) + b

# Define objective function
mean_sq_error = t.mean((lin_hypothesis - y)**2)

# Get the gradient 
gw, gb = t.grad(mean_sq_error, wrt=[w, b])

# Define the learning rate 
alpha = 0.001

# Define the update 
updates = (
    (w, w - alpha * gw), 
    (b, b - alpha * gb)
)


In [9]:
hyp = theano.function(inputs=[x], outputs=[lin_hypothesis])


train_function = theano.function(inputs=[x, y], 
                                 outputs=[lin_hypothesis, mean_sq_error], 
                                 updates=updates,
                                 name='train')

test_function = theano.function(inputs=[x, y],
                                outputs=[lin_hypothesis, mean_sq_error],
                                name='test')

## Train our Model 

In [10]:

for i in range(5):
    y_hat, error = train_function(train_x, train_y)
    print error



462.19456
1.52402930372e+11
7.13297972592e+19
3.33847916416e+28
1.56252275456e+37
[  9.21687612e+22   8.22553658e+22   7.73229543e+22   7.08648578e+22
   8.94320709e+22   8.37358408e+22   8.81524657e+22   8.00435141e+22
   9.43643821e+22   8.89946410e+22   8.94952287e+22   8.98927379e+22
   8.37836987e+22   1.06485558e+23   5.61283744e+22   6.67145637e+22
   5.99518652e+22   5.80911100e+22   6.55487194e+22   7.15588681e+22
   7.43807384e+22   6.26723470e+22   7.37121324e+22   8.18247412e+22
   7.27976916e+22   8.88279800e+22   5.18390648e+22   5.56102233e+22
   8.46646531e+22   7.79715084e+22   7.52701812e+22   8.51615130e+22
   8.76283300e+22   1.00234213e+23   9.72478463e+22   1.03307708e+23
   1.00004295e+23   1.13815208e+23   1.05876599e+23   9.40692254e+22
   1.02947294e+23   5.01554285e+22   5.14397329e+22   4.99217943e+22
   6.95265924e+22   9.19230633e+22   1.01869060e+23   8.30296603e+22
   8.92307230e+22   5.73095452e+22   5.60142778e+22   5.26042062e+22
   5.54687663e+22   6

## Test the model

In [ ]:
y_hat, test_error = test_function(test_x, test_y)
print y_hat
print test_err

# MLP 
## Set up Theano Variables  

In [48]:
x_mlp = t.dmatrix("X_mlp")
y_mlp = t.dvector("y_mlp")

fan_in = 7
n_nodes = 2

bound = np.sqrt(6. / (fan_in + n_nodes))

w1_value = np.asarray(
            np.random.uniform(
                low=-4*bound,
                high=4*bound,
                size=(fan_in, n_nodes)),
            dtype=theano.config.floatX
        )
b1_value = np.zeros(shape=n_nodes, dtype=theano.config.floatX)

w2_value = np.zeros(n_nodes, dtype=theano.config.floatX)
b2_value = np.asarray(0., dtype=theano.config.floatX)


w1 = theano.shared(value=w1_value, name="weights1")
b1 = theano.shared(value=b1_value, name="bias1")

w2 = theano.shared(value=w2_value, name="weights2")
b2 = theano.shared(value=b2_value, name="bias2")

params = [w1, b1, w2, b2]

In [49]:
# lin activation 
z = t.dot(x_mlp, w1) + b1
# non linear logistic activation - sigmoid i.e. activations/probabilites 
a = 1 / (1 + t.exp(-z)) 
# get linear output 
y_hat_hypothesis = t.dot(a, w2) + b2 

# Define objective function
mean_sq_error = t.mean((y_hat_hypothesis - y_mlp)**2)

# Get the gradient 
g_params = t.grad(mean_sq_error, wrt=params, add_names=True)

# Define the learning rate 
alpha = 0.01

# Define the update 
updates = []
for param, g_param in zip(params, g_params):
    updates.append((param, param - g_param * alpha))

In [50]:
train_function = theano.function(inputs=[x_mlp, y_mlp],
                                 outputs=[y_hat_hypothesis, mean_sq_error], 
                                 updates=updates)

test_function = theano.function(inputs=[x_mlp, y_mlp], 
                                outputs=[y_hat_hypothesis, mean_sq_error])

## Train the MLP

In [57]:
for i in range(2000):
    y_hat, error = train_function(train_x, train_y)
    if i%200 == 0:
        print error

41.780544
41.780544
41.780544
41.780544
41.780544
41.780544
41.780544
41.780544
41.780544
41.780544


## Test the MLP 

In [58]:
y_hat, error = test_function(test_x, test_y)